In [23]:
import os
import random
import yaml
import subprocess
import platform

# Define dataset directory
DIR_PATH = "slakh2100_flac_redux/reduced_test"

# Load metadata.yaml from a track folder
def load_metadata(track_path):
    yaml_path = os.path.join(track_path, "metadata.yaml")
    if not os.path.exists(yaml_path):
        return None
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

# Search stems based on either program_num or plugin_name
def find_matching_stems(metadata, program_num=None, plugin_name=None):
    if not metadata or "stems" not in metadata:
        return []
    return [
        (stem_id, stem_data)
        for stem_id, stem_data in metadata["stems"].items()
        if (program_num is not None and stem_data.get("program_num") == program_num) or
           (plugin_name is not None and stem_data.get("plugin_name") == plugin_name)
    ]

# Open a file or folder in the system's default file explorer
def open_file_explorer(path):
    if platform.system() == "Windows":
        os.startfile(path)
    elif platform.system() == "Darwin":  # macOS
        subprocess.run(["open", path])
    elif platform.system() == "Linux":
        subprocess.run(["xdg-open", path])
    else:
        print(f"Unsupported operating system: {platform.system()}")

# Find a random track and search for matching stems
def find_random_track(program_num=None, plugin_name=None):
    if program_num is None and plugin_name is None:
        print("You must specify either program_num or plugin_name.")
        return
    
    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return
    
    random.shuffle(track_folders)  # Shuffle to pick a random track
    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        metadata = load_metadata(track_path)
        matches = find_matching_stems(metadata, program_num, plugin_name)
        
        if matches:
            print(f"Track: {track}")
            for stem_id, stem_data in matches:
                print(f"  Stem: {stem_id}")
                print(f"    program_num: {stem_data.get('program_num', 'N/A')}")
                print(f"    plugin_name: {stem_data.get('plugin_name', 'N/A')}")
            
            # Open the track folder in the file explorer
            open_file_explorer(track_path)
            return
    
    print("No matching stems found.")

# find_random_track(program_num=6)
find_random_track(plugin_name="solo_guitar.nkm")

Track: Track01892
  Stem: S03
    program_num: 29
    plugin_name: solo_guitar.nkm


In [3]:
import os
import yaml

# Define dataset directory
DIR_PATH = "slakh2100_flac_redux/reduced_train"

# Load metadata.yaml from a track folder
def load_metadata(track_path):
    yaml_path = os.path.join(track_path, "metadata.yaml")
    if not os.path.exists(yaml_path):
        return None
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

# Iterate through the entire dataset and collect unique plugin names for a given class
def get_unique_plugins_by_class(inst_class):
    unique_plugins = set()
    
    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return
    
    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        metadata = load_metadata(track_path)
        
        if not metadata or "stems" not in metadata:
            continue
        
        for stem_data in metadata["stems"].values():
            # Cast program_num to integer before comparison
            if stem_data.get("inst_class") == inst_class:
                unique_plugins.add(stem_data.get("plugin_name", "Unknown"))
    
    print(f"[program_num: {inst_class}]")
    for plugin in sorted(unique_plugins):
        print(plugin)

# Example usage
get_unique_plugins_by_class("Guitar")

[program_num: Guitar]
/Library/Audio/Plug-Ins/Components/AGML2.component
elektrik_guitar.nkm
funk_guitar.nkm
harmonic_guitar.nkm
jazz_guitar.nkm
jazz_guitar2.nkm
jazz_guitar3.nkm
jazz_guitar4.nkm
nylon_guitar.nkm
nylon_guitar2.nkm
rhythm_rock_guitar.nkm
rock_guitar.nkm
solo_guitar.nkm
